## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
# tf.enable_eager_execution()

### Collect Data

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:

%cd /gdrive/My\ Drive/Deeplearning

/gdrive/My Drive/Deeplearning


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('prices.csv')

In [7]:
data.shape

(851264, 7)

### Check all columns in the dataset

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [0]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [0]:
data.describe()

,open,close,low,high,volume
count,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
mean,70.836986,70.857109,70.118414,71.543476,5.415113e+06
std,83.695876,83.689686,82.877294,84.465504,1.249468e+07
min,0.850000,0.860000,0.830000,0.880000,0.000000e+00
25%,33.840000,33.849998,33.480000,34.189999,1.221500e+06
50%,52.770000,52.799999,52.230000,53.310001,2.476250e+06
75%,79.879997,79.889999,79.110001,80.610001,5.222500e+06
max,1584.439941,1578.130005,1549.939941,1600.930054,8.596434e+08


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'], axis =1, inplace=True)

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data_df = data.loc[0:999,:]

In [11]:
data_df.shape

(1000, 5)

In [8]:
data_df['volume'] = data_df['volume']/1000000
# data_df.loc[:, 'volume'] =  data_df['volume']*1000000

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
data_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:

import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

X = data_df.drop(['volume'], axis =1)
Y = data_df['volume']

# Convert X, Y to ndarray of type float 
X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32)

# L2 norm on axis 1 
X_normal = normalize(X)

Xtrain,Xtest,ytrain,ytest=train_test_split(X_normal,Y,test_size=0.3,random_state=7)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
type(Xtrain)

numpy.ndarray

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
Xtrain

array([[0.50021227, 0.50035977, 0.49549329, 0.50389898],
       [0.5023256 , 0.50103615, 0.4931476 , 0.50342548],
       [0.50043012, 0.49899534, 0.49628512, 0.50425631],
       ...,
       [0.50019286, 0.49977148, 0.49766452, 0.50236003],
       [0.50253592, 0.49852157, 0.49633898, 0.50257488],
       [0.50121458, 0.49876764, 0.49595365, 0.50402853]])

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
# x_n = tf.nn.l2_normalize(x,1)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

In [0]:
# W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
# b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

w = tf.random.uniform(shape=[4,1], dtype=tf.float32)
b = tf.zeros(shape=[1], dtype=tf.float32)

2.Define a function to calculate prediction

In [0]:
w

<tf.Tensor 'zeros_10:0' shape=(4, 1) dtype=float32>

3.Loss (Cost) Function [Mean square error]

In [0]:
# loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

In [0]:
def prediction(x,w,b):
  mx = tf.matmul(x,w)
  y = tf.add(mx,b)
  return y

def loss (y, predict):
  return tf.reduce_mean(tf.square(y-predict))

def train(x,y_actual, w,b, learning_rate=0.01):
  with tf.GradientTape() as t:
    t.watch([w,b])
    current_prediction = prediction(x,w,b)
    current_loss = loss(y_actual, current_prediction)
  dw,db = t.gradient(current_loss, [w,b])
#   print(dw.eval(session=sess), 'dwwww', db.eval(session=sess))
  w = w-learning_rate*dw
  b = b-learning_rate*db
  return w,b
    

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
# train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

In [0]:
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100


## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [38]:
with sess.as_default():
  for i in range(100):
    w,b = train(Xtrain, ytrain,w,b)
    train_loss = loss(ytrain, prediction(Xtrain, w,b))
    if i%5 == 0:
      print ('Training loss at step: ', i, ' is ', train_loss.eval())

('Training loss at step: ', 0, ' is ', 292.04886)
('Training loss at step: ', 5, ' is ', 293.7989)
('Training loss at step: ', 10, ' is ', 294.01932)
('Training loss at step: ', 15, ' is ', 289.2111)
('Training loss at step: ', 20, ' is ', 297.1347)
('Training loss at step: ', 25, ' is ', 293.64703)
('Training loss at step: ', 30, ' is ', 294.67657)
('Training loss at step: ', 35, ' is ', 287.07358)
('Training loss at step: ', 40, ' is ', 298.78482)
('Training loss at step: ', 45, ' is ', 297.18073)
('Training loss at step: ', 50, ' is ', 296.2932)
('Training loss at step: ', 55, ' is ', 298.7561)
('Training loss at step: ', 60, ' is ', 294.25693)
('Training loss at step: ', 65, ' is ', 294.63757)
('Training loss at step: ', 70, ' is ', 292.61478)
('Training loss at step: ', 75, ' is ', 294.96304)
('Training loss at step: ', 80, ' is ', 294.88748)
('Training loss at step: ', 85, ' is ', 294.95145)
('Training loss at step: ', 90, ' is ', 290.8941)
('Training loss at step: ', 95, ' is ',

In [39]:
w.eval(session=sess)

array([[0.9886594 ],
       [0.6902971 ],
       [0.55058527],
       [0.6236739 ]], dtype=float32)

In [40]:
b.eval(session=sess)

array([0.03572178], dtype=float32)

In [87]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

In [88]:
b.shape

TensorShape([Dimension(1)])

In [0]:
# loss = tf.reduce_mean(tf.square(y-y_),name='Loss')
# train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)
# for epoch in range(training_epochs):
            
#     #Calculate train_op and loss
#     _, train_loss = sess.run([train_op,loss],feed_dict={x:features, y_:actual_prices})
    
#     if epoch % 10 == 0:
#         print ('Training loss at step: ', epoch, ' is ', train_loss.eval())
        

### Get the shapes and values of W and b

In [25]:
w.eval(session=sess)

array([[0.29808223],
       [0.25640762],
       [0.20740473],
       [0.40695703]], dtype=float32)

In [26]:
b.eval(session=sess)

array([0.], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [0]:
y_pred=prediction(Xtest,w,b)

# prediction(Xtest,w,b).eval(session=sess)

In [101]:
loss(y_pred,ytest).eval(session=sess)



20948.188

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_df = pd.read_csv('11_Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [44]:
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
# from sklearn.preprocessing import OneHotEncoder
# iris_df_copy = OneHotEncoder().fit_transform(iris_df)

iris_df_new = pd.get_dummies(iris_df)

In [80]:
iris_df_new.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X = iris_df_new.drop(['Species_Iris-versicolor', 'Species_Iris-virginica','Species_Iris-setosa'	, 'Id'], axis=1)
Y = iris_df_new[['Species_Iris-versicolor', 'Species_Iris-virginica','Species_Iris-setosa']]

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
from keras.models import Sequential
from keras.layers import Dense


model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


### Model Training 

In [85]:
model.fit(X, Y, epochs=100, batch_size=10)

Epoch 1/100
150/150 [==============================] - 4s 24ms/step - loss: 1.0011 - acc: 0.4800
Epoch 2/100
150/150 [==============================] - 0s 146us/step - loss: 0.9698 - acc: 0.4333
Epoch 3/100
150/150 [==============================] - 0s 138us/step - loss: 0.9462 - acc: 0.4133
Epoch 4/100
150/150 [==============================] - 0s 142us/step - loss: 0.9223 - acc: 0.4800
Epoch 5/100
150/150 [==============================] - 0s 135us/step - loss: 0.8944 - acc: 0.3933
Epoch 6/100
150/150 [==============================] - 0s 135us/step - loss: 0.8763 - acc: 0.5067
Epoch 7/100
150/150 [==============================] - 0s 137us/step - loss: 0.8519 - acc: 0.4533
Epoch 8/100
150/150 [==============================] - 0s 139us/step - loss: 0.8286 - acc: 0.5400
Epoch 9/100
150/150 [==============================] - 0s 143us/step - loss: 0.8141 - acc: 0.4933
Epoch 10/100
150/150 [==============================] - 0s 154us/step - loss: 0.7940 - acc: 0.5067
Epoch 11/100
150/150

### Model Prediction

In [86]:
# evaluate the model
scores = model.evaluate(X, Y)
scores[1]*100

150/150 [==============================] - 2s 10ms/step


96.6666668256124

### Save the Model

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?